In [41]:
from langchain.chat_models import ChatOllama, ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler


PRIVATE = False

# openai 결제를 하지 않아서 ollama를 사용하였습니다.
model = "mistral:latest" if PRIVATE else "gpt-3.5-turbo"
ChatModel = ChatOllama if PRIVATE else ChatOpenAI

llm = ChatModel(
    model=model,
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [42]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationSummaryBufferMemory

examples = [
    {
        "question": "Top Gun: Maverick",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "question": "The Godfather",
        "answer": "👨‍👨‍👦🔫🍝",
    },
    {
        "question": "Avatar",
        "answer": "🌿🌌👽",
    },
    {
        "question": "Jurassic World",
        "answer": "🦖🌴🌋",
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}"),
    ]
)


example_template = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

full_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a helpful AI assistant for movie summary with 3 emoji.",
        ),
        MessagesPlaceholder(variable_name="history"),
        example_template,
        ("human", "{question}"),
    ]
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | full_prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )

In [43]:
invoke_chain("Iron Man")

🔥🦾💥

In [44]:
invoke_chain("Frozen")

❄️👑⛄

In [45]:
invoke_chain("내가 처음에 질문한 영화 제목이 뭐야?")

Iron Man